In [1]:
# Intsall transformers
!pip install transformers


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import Libraries 
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

Load Summarization Pipeline

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


c:\Users\acer\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\acer\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Get a blog

In [4]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"

In [5]:
r = requests.get(URL)

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [7]:
ARTICLE

'Sign up Sign In Sign up Sign In Member-only story A Bayesian Take On Model Regularization Ryan Sander Follow Towards Data Science -- 1 Listen Share I’m currently reading “How We Learn” by Stanislas Dehaene. First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI. One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i.e. given only a limited amount of experience¹. One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world. In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models. But why do we do this, even from birth¹? One argument is that, contrary to the frequentist view in child psychology (the belief that babi

Create Chunks

In [8]:
max_chunk = 500

In [9]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [10]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [11]:
len(chunks)


4

In [14]:
len(chunks[1].split(' '))

491

Summarize Text 

In [15]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [16]:
res[0]

{'summary_text': ' In machine learning, regularization, or model complexity control, is an essential and common practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models . The Bayesian approach is called analytic regularization for supervised learning .'}

In [17]:
res[1]

{'summary_text': ' We will analyze these claims for regression problems, but they extend to other supervised learning tasks, such as classification, as well . We will focus on rigorously presenting the mathematics behind these claims . In the next section, we will use Bayes’ Rule to derive our L2 regularized estimator .'}

In [18]:
' '.join([summ['summary_text'] for summ in res])

' In machine learning, regularization, or model complexity control, is an essential and common practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models . The Bayesian approach is called analytic regularization for supervised learning .  We will analyze these claims for regression problems, but they extend to other supervised learning tasks, such as classification, as well . We will focus on rigorously presenting the mathematics behind these claims . In the next section, we will use Bayes’ Rule to derive our L2 regularized estimator .  Using Bayes’ Rule, we can show that the mean and mode of the posterior distribution of w is the solution for LASSO regression when we invoke a Gaussian prior distribution on w . We’ll now examine a similar case with a Laplace prior . Here is the corresponding derivation for Lasso: (Note that in the last step, we set p = 1, q = 1)  A huge thank you to CODECOGS for their online 

In [19]:

text = ' '.join([summ['summary_text'] for summ in res])

Save output to text file

In [20]:
with open('summary.txt', 'w') as f:
    f.write(text)